In [ ]:
import openai
import os
import csv
import librosa
import numpy as np

def transcribe_audio(api_key, audio_file):
    client = openai.OpenAI(api_key=api_key)

    with open(audio_file, "rb") as file:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=file,
            response_format="verbose_json"
        )

    return response

def compute_pitch_variation(audio_file):
    """Compute the variation in pitch to analyze expressive use of tone."""
    y, sr = librosa.load(audio_file, sr=None)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)

    # Extract pitch values where magnitude is above a threshold
    pitch_values = pitches[magnitudes > np.median(magnitudes)]

    if len(pitch_values) == 0:
        return 0  # No pitch variations detected

    pitch_variation = np.std(pitch_values)  # Standard deviation of pitch values
    return pitch_variation

def save_to_csv(results, output_file="/content/output_folder/output.csv"):
    """Save pitch variations to a CSV file."""
    os.makedirs("/content/output_folder", exist_ok=True)

    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["File Name", "Pitch Variation"])
        writer.writerows(results)

def process_multiple_files(api_key, audio_files, output_file):
    results = []

    for audio_file in audio_files:
        print(f"Processing {audio_file}...")
        transcribe_audio(api_key, audio_file)  # Using Whisper API
        pitch_variation = compute_pitch_variation(audio_file)
        results.append([os.path.basename(audio_file), pitch_variation])

    save_to_csv(results, output_file)
    print(f"All data saved to {output_file}")

if __name__ == "__main__":
    API_KEY = ""
  # Replace with your OpenAI API key
    AUDIO_FILES = [
        "/content/ABB_segment_2 (1).mp3",
        "/content/Adani Wilmar Limited (NSEI_AWL) Jan-31-2024 - Audio_segment_2.mp3",
        "/content/Adani Wilmar Limited (NSEI_AWL) Jul-30-2024 - Audio_segment_2.mp3"
    ]  # Replace with your list of audio files
    OUTPUT_FILE = "/content/output_folder/output.csv"

    process_multiple_files(API_KEY, AUDIO_FILES, OUTPUT_FILE)


Processing /content/ABB_segment_2 (1).mp3...
Processing /content/Adani Wilmar Limited (NSEI_AWL) Jan-31-2024 - Audio_segment_2.mp3...
Processing /content/Adani Wilmar Limited (NSEI_AWL) Jul-30-2024 - Audio_segment_2.mp3...
All data saved to /content/output_folder/output.csv
